In [2]:
import csv
import pandas as pd


# Comparing Interests Engagement Over Time

Top topics: sports, health, entertainment, food, politics, education, history, environment, science, climate, inequality, technology


# Cleaning and Prepping the Data

In [5]:

# Sample data
df1 = pd.read_csv("../Datasets/Master_Pixstory_ANALYSIS_Dataset.tsv", sep='\t')
df1 = df1.drop(columns=[col for col in df1.columns if col not in ['Interest', 'Date (No Timestamp)']])
df1 = df1.dropna(subset=['Interest'])

# Convert the 'date' column to a string
df1['Date (No Timestamp)'] = df1['Date (No Timestamp)'].apply(lambda x: str(x))

# Rename the 'date' column to 'Date'
df1 = df1.rename(columns={'Date (No Timestamp)': 'Date'})

# Convert all words in the "Interest" column to lowercase
df1['Interest'] = df1['Interest'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)
df1['Interest'] = df1['Interest'].apply(lambda x: [word.lower() for word in x])


/var/folders/pj/c3bwxcz17fx4v5_bp1h430h00000gn/T/ipykernel_29553/1846110628.py:2: DtypeWarning: Columns (0,2,5,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("../Datasets/Master_Pixstory_ANALYSIS_Dataset.tsv", sep='\t')


# Counting frequency of Interests on Daily Basis

In [10]:
# Create a function to tally the counts of each interest
def tally_interests(interests, interest_type):
    counts = []
    for interest in interests:
        count = 0
        for word in interest:
            if word == interest_type:
                count += 1
        counts.append(count)
    return counts

# Group by date and tally interests
df2 = df1.groupby('Date').agg(
    sports_count=('Interest', lambda x: sum(tally_interests(x, 'sports'))),
    health_count=('Interest', lambda x: sum(tally_interests(x, 'health'))),
    entertainment_count=('Interest', lambda x: sum(tally_interests(x, 'entertainment'))),
    food_count=('Interest', lambda x: sum(tally_interests(x, 'food'))),
    politics_count=('Interest', lambda x: sum(tally_interests(x, 'politics'))),
    education_count=('Interest', lambda x: sum(tally_interests(x, 'education')))
).reset_index()

# Convert the "Date" column to datetime objects, dropping rows with invalid values
df2 = df2[pd.to_datetime(df2['Date'], format='%Y-%m-%d', errors='coerce').notnull()]

# Sort df2 by increasing date
df2 = df2.sort_values('Date', ascending=True)

# Calculate the cumulative count for each interest type
df2['sports_count_cumulative'] = df2['sports_count'].cumsum()
df2['health_count_cumulative'] = df2['health_count'].cumsum()
df2['entertainment_count_cumulative'] = df2['entertainment_count'].cumsum()
df2['food_count_cumulative'] = df2['food_count'].cumsum()
df2['politics_count_cumulative'] = df2['politics_count'].cumsum()
df2['education_count_cumulative'] = df2['education_count'].cumsum()


df2

,Date,sports_count,health_count,entertainment_count,food_count,politics_count,education_count,sports_count_cumulative,health_count_cumulative,entertainment_count_cumulative,food_count_cumulative,politics_count_cumulative,education_count_cumulative
1,2020-01-12,1,1,1,2,1,1,1,1,1,2,1,1
2,2020-02-12,0,0,0,0,0,0,1,1,1,2,1,1
3,2020-11-17,345,489,315,263,403,279,346,490,316,265,404,280
4,2020-11-18,25,28,17,17,14,14,371,518,333,282,418,294
5,2020-11-19,44,30,30,36,32,28,415,548,363,318,450,322
...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,2022-11-08,0,0,0,1,0,0,24489,21749,22919,21458,21854,20927
508,2022-12-01,4,4,4,4,4,4,24493,21753,22923,21462,21858,20931
509,2022-12-05,0,0,0,0,0,0,24493,21753,22923,21462,21858,20931
510,2022-12-07,0,0,2,0,0,0,24493,21753,22925,21462,21858,20931


# Reformatting Data for Observable

In [12]:
# Create a new dataframe for Observable
df_observable = pd.DataFrame(columns=['date', 'name', 'category', 'value'])

# Loop over the interest type columns and populate the new dataframe
for col in ['sports_count_cumulative', 'health_count_cumulative', 'entertainment_count_cumulative', 'food_count_cumulative', 'politics_count_cumulative', 'education_count_cumulative']:
    df_observable = df_observable.append(pd.DataFrame({
        'date': df2['Date'],
        'name': col.split('_')[0].capitalize(),
        'category': 'Interest',
        'value': df2[col]
    }))

# Reset the index of the new dataframe
df_observable = df_observable.reset_index(drop=True)

# Sort df2 by increasing date
df_observable = df_observable.sort_values('date', ascending=True)

df_observable



/var/folders/pj/c3bwxcz17fx4v5_bp1h430h00000gn/T/ipykernel_29553/3761071893.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_observable = df_observable.append(pd.DataFrame({
/var/folders/pj/c3bwxcz17fx4v5_bp1h430h00000gn/T/ipykernel_29553/3761071893.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_observable = df_observable.append(pd.DataFrame({
/var/folders/pj/c3bwxcz17fx4v5_bp1h430h00000gn/T/ipykernel_29553/3761071893.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_observable = df_observable.append(pd.DataFrame({
/var/folders/pj/c3bwxcz17fx4v5_bp1h430h00000gn/T/ipykernel_29553/3761071893.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

,date,name,category,value
0,2020-01-12,Sports,Interest,1
1022,2020-01-12,Entertainment,Interest,1
2044,2020-01-12,Politics,Interest,1
511,2020-01-12,Health,Interest,1
2555,2020-01-12,Education,Interest,1
...,...,...,...,...
2043,2022-12-08,Food,Interest,21465
1021,2022-12-08,Health,Interest,21753
510,2022-12-08,Sports,Interest,24493
2554,2022-12-08,Politics,Interest,21858


In [13]:
# Save the DataFrame as a CSV file
df_observable.to_csv('pixstory_test3.csv', index=False)